In [ ]:
# -*- coding: utf-8 -*-
'''
orca-uv Jupyter edition

1. Upload 'orca.out' to the same folder
2. Insert the filename here: imp_data('orca.out')
3. Adjust the plot; use the interactive Matplotlib to select a specific region
4. Save the spectrum by clicking on the 'disk' symbol of the interactive Matplotlib window

'''

# the matplotlip widget in IPython 
%matplotlib widget

import re                                      # regular expressions
import numpy as np                             # summation
import matplotlib.pyplot as plt                # plots
from scipy.signal import find_peaks            # peak detection
from ipywidgets import interactive, HBox, VBox # interactive ui elements

def imp_data(file):
    specstring_start = 'ABSORPTION SPECTRUM VIA TRANSITION ELECTRIC DIPOLE MOMENTS'          #check orca.out from here
    specstring_end = 'ABSORPTION SPECTRUM VIA TRANSITION VELOCITY DIPOLE MOMENTS'            #stop reading orca.out from here
    # 3 empty lists (first one is not really necessary)
    statelist=list()            #mode
    energylist=list()           #energy cm-1
    intenslist=list()           #fosc
    try:
        with open(file, "r") as input_file:
            for line in input_file:
                #start exctract text 
                if specstring_start in line:
                #found UV data in orca.out
                    found_uv_section=True
                    for line in input_file:
                        #stop exctract text 
                        if specstring_end in line:
                            break
                        #only recognize lines that start with number
                        #split line into 3 lists mode, energy, intensities
                        #line should start with a number
                        if re.search("\d\s{1,}\d",line): 
                            statelist.append(int(line.strip().split()[0])) 
                            energylist.append(float(line.strip().split()[1]))
                            intenslist.append(float(line.strip().split()[3]))
    # file not found -> message
    except FileNotFoundError:
        print(f"'{file}'" + " not found")
    if statelist:
        return statelist, energylist, intenslist
    # Abs part not found -> message
    else:
        print(f"'{specstring_start}'" + " not found in" + f"'{file}'")

def roundup(x):
    #round to next 10 or 100
    if nm_plot:
        return x if x % 10 == 0 else x + 10 - x % 10
    else:
        return x if x % 100 == 0 else x + 100 - x % 100

def rounddown(x):
    #round to next 10 or 100
    if nm_plot:
        return x if x % 10 == 0 else x - 10 - x % 10
    else:
        return x if x % 100 == 0 else x - 100 - x % 100

def gauss(a,m,x,w):
    # calculation of the Gaussian line shape
    # a = amplitude (max y, intensity)
    # x = position
    # m = maximum/meadian (stick position in x, wave number)
    # w = line width, FWHM
    return a*np.exp(-(np.log(2)*((m-x)/w)**2))

def plot_abs(w_nm = 15,
             w_wn = 1000,
             nm_wn = True, 
             show_stick = True, 
             label_peaks = True,
             show_grid = False,
             show_single_gauss = False):
    
    # w_nm = line width nm 
    # w_wn = line width wn
    # show_stick = show the stick spectrum
    # labe_peaks = label the peaks
    # show_grid = show a grid
    # show_single_gauss = show the single gauss functions

    # always close before draw a new one
    plt.close()

    # list for sum of single gauss functions
    gauss_sum = []
    # position
    energylist = abs_data[1]
    # intensity
    intenslist = abs_data[2]
    
    # to tell the round functions if it is nm or wn
    global nm_plot
    
    # convert wn -> nm, change x-axis label and set nm_plot = True
    if nm_wn:
        w = w_nm
        energylist = [1/wn*10**7 for wn in energylist]
        nm_plot = True       
        x_label = r'$\lambda$ /nm' 
    # change x-axis label and set wn_plot = True
    else:
        w = w_wn
        nm_plot = False
        x_label = r'energy /cm$^{-1}$' 
     
    # prepare plot
    fig, ax = plt.subplots()
    # plot range in x, start from 0 for peak detection
    plt_range_x=np.arange(0, max(energylist) + w*3, 1)

    # plot single gauss function for every stick x-value if TRUE
    # generate summation of single gauss functions
    for index, wn in enumerate(energylist):
        if show_single_gauss:
            ax.plot(plt_range_x,gauss(intenslist[index], plt_range_x, wn, w), alpha=0.5) 
            ax.fill_between(plt_range_x,gauss(intenslist[index], plt_range_x, wn, w), alpha=0.5)
        gauss_sum.append(gauss(intenslist[index], plt_range_x, wn, w))
    
    # y values of the gauss summation
    plt_range_gauss_sum_y = np.sum(gauss_sum, axis=0)
    
    # find peaks scipy function, change height for level of detection
    peaks , _ = find_peaks(plt_range_gauss_sum_y, height = 0)

    # plot the spectrum
    ax.plot(plt_range_x, plt_range_gauss_sum_y, color="black", linewidth=0.8)

    # show stick spectrum if TRUE
    if show_stick:
        ax.stem(energylist, intenslist, linefmt="dimgrey", markerfmt=" ", basefmt=" ")

    # title, label, ticks, linear locator
    ax.set_title('Absorption spectrum', fontweight = 'bold')
    ax.set_xlabel(x_label)  
    ax.get_yaxis().set_ticks([])   
    ax.minorticks_on()
    ax.xaxis.set_major_locator(plt.LinearLocator())

    # set ylim
    plt.ylim(0, max(plt_range_gauss_sum_y) + max(plt_range_gauss_sum_y) * 0.1)
    ax.set_ylabel('intensity')
    label_rel_pos_y=5

    # show grid if TRUE
    if show_grid:
        ax.grid(True,which='major', axis='x',color='black',linestyle='dotted', linewidth=0.5)

    # show labels on peaks if TRUE
    if label_peaks: 
        for index, txt in enumerate(peaks):
            ax.annotate(peaks[index],xy=(peaks[index],plt_range_gauss_sum_y[peaks[index]]),ha="center",rotation=90,size=8,
                xytext=(0,5), textcoords='offset points')

    # auto x
    xlim_autostart = rounddown(min(energylist) - w*3) #startx from data
    xlim_autoend = roundup(max(plt_range_x)) #auto endx from data

    #x should not below zero - x-axis range
    if xlim_autostart < 0:
        plt.xlim(0,xlim_autoend) 
    else:
        plt.xlim(xlim_autostart,xlim_autoend)
        
    #y-axis range - dynamic y range
    xmin=int(ax.get_xlim()[0]) #get recent xlim min
    xmax=int(ax.get_xlim()[1]) #get recent xlim max
    if xmin > xmax:
        ymax=max(plt_range_gauss_sum_y[xmax:xmin])  #get y max from recent x range for inverted x axis
        ax.set_ylim(0,ymax+ymax*0.1)                # +10% for labels
    elif xmax > xmin:
        ymax=max(plt_range_gauss_sum_y[xmin:xmax])  #get y max from recent x range for normal x axis
        ax.set_ylim(0,ymax+ymax*0.1)                # +10% for labels
    
    # resize the Matplotlib window
    N = 1.5
    params = plt.gcf()
    plSize = params.get_size_inches()
    params.set_size_inches((plSize[0]*N, plSize[1]*N))

    # tight layout and show
    plt.tight_layout() 
    plt.show()


# upload orca.out in the same folder (location)
# insert the filename; imp_data('orca.out)
abs_data = imp_data('1e.out')

# make it interactive
interactive_plot = interactive(plot_abs, 
                               w_nm = (5, 150, 5), 
                               w_wn = (100, 5000, 100), 
                               show_stick = True, 
                               nm_wn = True, 
                               label_peaks = True,
                               show_grid = False,
                               show_single_gauss = False,
                               )

# ui elements (all to last -1)
ui_el = interactive_plot.children[:-1]
# plot window (last)
plot_win = interactive_plot.children[-1]

# ui elements in rows
row1 = HBox(ui_el[:2])
row2 = HBox(ui_el[2:4])
row3 = HBox(ui_el[4:6])
row4 = HBox(ui_el[6:])

# display
interactive_plot = VBox([row1, row2, row3, row4, plot_win])
interactive_plot
